In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
plt.ion()

import sys
from ark.utils.metacluster_remap_gui import MetaClusterData, MetaClusterGui

## Launching
I have added jupyerlab to the requirements file because it gives more horizontal space to use the GUI, to try this for yourself, after starting jupyter you can change the url to `lab` instead of `tree`.

e.g. `http://127.0.0.1:8888/tree/` -> `http://127.0.0.1:8888/lab/`

## Usage
- To select one or more clusters, click anywhere on either heatmap. Clicking a selected cluster will deselect it.
- To remap the selected clusters, right click anywhere on either heatmap.
- The bars at the top show the relative size of each cluster.
- If you remap all cluster in a metacluster, the metacluster is now *gone*
- You can click new metacluster to add current selection to a new metacluster
- You final mapping will be in an output file in the same directory as the input files

In [2]:
data = MetaClusterData("../data/example_dataset/metaclustering", "ex1_clusters_nozscore.csv", "ex1_clusters_pixelcount.csv", "ex1_output_mapping.csv")
mcg = MetaClusterGui(data, heatmapcolors='viridis', width=17, debug=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

FloatSlider(value=3.0, description='Max Zscore:', max=10.0, min=1.0, readout_format='.1f', step=0.5)

Button(button_style='warning', description='Clear Selection', icon='ban', style=ButtonStyle(), tooltip='Clear …

Button(button_style='success', description='New metacluster', icon='plus', style=ButtonStyle(), tooltip='Creat…

Output(layout=Layout(border='1px solid black'), outputs=({'output_type': 'stream', 'name': 'stdout', 'text': '…

# Developers Maintanence Guide

This guide is outline/work in progress

High level Architecture

- MetaClusterData - Data IO and state maintence
- MetaClusterGui - Handle the 

This metacluster remapper GUI uses the ipywidgets backend for Matplotlib. Addition widgets are used for extra input/output.

How to debug callbacks
https://ipywidgets.readthedocs.io/en/latest/examples/Output%20Widget.html#Debugging-errors-in-callbacks-with-the-output-widget

In [3]:
# Save out current notebook
!cp ./example_manually_adjust_metaclusters.ipynb ../data/example_dataset/metaclustering/example_manually_adjust_metaclusters.ipynb